In [105]:
import glob
import os
import warnings
import mlflow
import mlflow.pytorch 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom
from pydicom.data import get_testdata_files
from sklearn.model_selection import train_test_split
from torchvision import transforms
from IPython.display import Markdown, display

 # Import functions from the module
import importlib
import help_files._0_definitions 
import  help_files._1_visuals_script
# import  help_files._01_load_data
 # Reload the module to apply the changes to the script
importlib.reload(help_files._0_definitions)
importlib.reload(help_files._1_visuals_script)
# importlib.reload(help_files._01_load_data)
import  help_files._1_visuals_script  as pauls_vs
# Group by 'condition', 'level', and 'severity' and count occurrences
from help_files._0_definitions import count_severity_by_condition_level 
# Define the path
from pathlib import Path

pd.set_option("display.width", 1000)  # Set a large width to prevent line wrapping
 

In [106]:
### In definitions are all the functions that are used in the notebook and globals
with open("help_files/_0_definitions.py") as file:
    exec(file.read())
    ### In definitions are all the functions that are used in the notebook and globals
with open("help_files/_0_run_definitions.py") as file:
    exec(file.read())

In [107]:
# loading data
file_names = ["train_df_3_cat.csv", "test_df_3_cat.csv"]
# Load the data from the CSV files
dataframes = [pd.read_csv(data_path_vor / file_name) for file_name in file_names]
# Unpack the dataframes into separate variables
train_df, test_df = dataframes

print("DataFrames have been loaded successfully.")


DataFrames have been loaded successfully.


In [108]:
# # end sample or small sample    
if whole_data_set == True:
    print("Using the whole data set")
else:
    train_df = train_df.sample(n=20, random_state=RSEED)
    test_df = test_df.sample(n=20, random_state=RSEED)
    display(Markdown('<span style="color:red"> this is a small sample : 48692</span>'))

<span style="color:red"> this is a small sample : 48692</span>

Calss definition dataloader (do not change over models)

In [109]:
import pydicom
from torchvision import transforms
from PIL import Image
import torch
from torch.utils.data import Dataset
import numpy as np

class MRILocalizationDataset(Dataset):
    def __init__(self, data, transform=None, has_coordinates=True):
        """
        Args:
            data (pd.DataFrame): DataFrame containing image paths and other info.
            transform (callable, optional): Optional transform to be applied on a sample.
            has_coordinates (bool, optional): Whether the dataset includes x, y coordinates.
        """
        self.data = data
        self.transform = transform
        self.has_coordinates = has_coordinates

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Get the image path
        img_path = self.data.iloc[idx]['image_path']
        
        # Read the DICOM file
        dicom_image = pydicom.dcmread(img_path)
        
        # Convert the DICOM pixel data to a NumPy array
        image_array = dicom_image.pixel_array
        
        # Convert NumPy array to PIL Image
        image = Image.fromarray(image_array)
        
        # Convert to RGB if necessary (some DICOM files might be grayscale)
        if image.mode != 'RGB':
            image = image.convert('RGB')

        # Apply transformations (e.g., resizing, normalization)
        if self.transform:
            image = self.transform(image)
        
        # Return image with or without coordinates
        if self.has_coordinates:
            x = torch.tensor(self.data.iloc[idx]['x'], dtype=torch.float32)
            y = torch.tensor(self.data.iloc[idx]['y'], dtype=torch.float32)
            return image, torch.tensor([x, y])
        else:
            return image





training

In [110]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import models
# %pip install keras
# %pip install tensorflow
# from tensorflow.keras.callbacks import EarlyStopping  # Import EarlyStopping
import numpy as np  # Import numpy for setting the random seed

# Define the transformation to be applied to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to the expected input size of the model
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization
])

# Create the dataset
dataset = MRILocalizationDataset(data=train_df, transform=transform)

# Split the dataset into training and validation sets (80% train, 20% validation)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

batch_size = 256
# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Load ResNet-50 and set up for classification
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = train_df['severity'].nunique()

model = models.resnet50(weights='IMAGENET1K_V1')
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.to(device)

# Define loss functions and optimizer
learining_rate = 0.0001 
criterion_cel = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learining_rate)
num_epochs = 20

# Early stopping parameters
# Lists to store loss values for plotting
train_losses_cel = []
val_losses_cel = []

# Early stopping parameters
diverge_count = 0              # Counter for divergence-based stopping
stop_threshold = 0.3           # Threshold for divergence
max_diverge_count = 3          # Max number of epochs with diverging validation loss

patience_counter = 0           # Counter for plateau-based stopping    
patience = 5                   # Number of epochs for validation loss plateau


# Calculate number of layers in the model
num_layers = len(list(model.children()))


In [111]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import models, transforms
import mlflow
import mlflow.pytorch
import os


train_dataset = MRILocalizationDataset(data=train_df, transform=transform, has_coordinates=True)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# Localization model (based on ResNet-50)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.resnet50(weights='IMAGENET1K_V1')
model.fc = nn.Linear(model.fc.in_features, 2)  # Output x and y coordinates
model.to(device)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
num_epochs = 20

# Early stopping parameters
best_val_loss = float('inf')  # Initialize best validation loss
patience_counter = 0  # Counter for plateau-based stopping    
patience = 5  # Number of epochs for validation loss plateau

# MLflow experiment setup
experiment_name = "MRI_Localization_ResNet50"
mlflow.set_experiment(experiment_name)

# Training loop with MLflow tracking
with mlflow.start_run():
    mlflow.log_param("learning_rate", 0.0001)
    mlflow.log_param("optimizer", "Adam")
    mlflow.log_param("batch_size", 4)
    mlflow.log_param("num_epochs", num_epochs)
    mlflow.log_param("model_architecture", "ResNet-50 for Localization")
    mlflow.log_param("output_coordinates", "x, y")

       # Set descriptive tags for the model
    mlflow.set_tag("model_description", "ResNet-50 for 3 localization and Sagittal T2/STIR and Sagittal T1 images")

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        # Training phase
        for images, coordinates in train_loader:
            images, coordinates = images.to(device), coordinates.to(device)

            # Zero gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, coordinates)
            running_loss += loss.item()

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

        epoch_loss = running_loss / len(train_loader)
        mlflow.log_metric("train_loss", epoch_loss, step=epoch)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

        # Validation phase (if you have a validation set, add validation loss calculation here)
        # Assuming val_loader is defined and you have a validation dataset
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, coordinates in val_loader:
                images, coordinates = images.to(device), coordinates.to(device)
                outputs = model(images)
                loss = criterion(outputs, coordinates)
                val_loss += loss.item()
        
        val_loss = val_loss / len(val_loader)
        mlflow.log_metric("val_loss", val_loss, step=epoch)
        print(f"Validation Loss: {val_loss:.4f}")

        # Check if validation loss improves and save best model
        if val_loss > epoch_loss * (1 + stop_threshold):
            diverge_count += 1
            if diverge_count >= max_diverge_count:
                print(f"Early stopping at epoch {epoch + 1} due to validation loss diverging.")
                break
        else:
            diverge_count = 0  # Reset diverge count if validation loss is not diverging

        # Check for early stopping (plateau-based stopping)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0  # Reset plateau counter if validation loss improves
            # Save the model weights when validation loss improves (best model)
            best_model_weight_path = f"best_localization_model_epoch_{epoch + 1}.pt"
            torch.save(model.state_dict(), best_model_weight_path)
            mlflow.log_artifact(best_model_weight_path)  # Log the best model weights
            os.remove(best_model_weight_path)  # Optionally, delete the local file after logging
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch + 1} due to lack of validation loss improvement.")
                break

    # Log final model
    mlflow.pytorch.log_model(model, "localization_model")

print("Training complete!")


Epoch [1/20], Loss: 114567.6531
Validation Loss: 93781.2422
Epoch [2/20], Loss: 114090.3094
Validation Loss: 93321.2969
Epoch [3/20], Loss: 113556.3891
Validation Loss: 92919.6250
Epoch [4/20], Loss: 113019.4875
Validation Loss: 92833.6406
Epoch [5/20], Loss: 112316.6578
Validation Loss: 92819.1484
Epoch [6/20], Loss: 111656.4656
Validation Loss: 91821.4297
Epoch [7/20], Loss: 110981.6781
Validation Loss: 90653.6875
Epoch [8/20], Loss: 110043.0703
Validation Loss: 89232.7891
Epoch [9/20], Loss: 109436.6359
Validation Loss: 88826.1953
Epoch [10/20], Loss: 108617.9453
Validation Loss: 88864.7188
Epoch [11/20], Loss: 108665.6086
Validation Loss: 86746.1406
Epoch [12/20], Loss: 107543.1672
Validation Loss: 84165.9844
Epoch [13/20], Loss: 106835.7734
Validation Loss: 82990.0156
Epoch [14/20], Loss: 106634.7039
Validation Loss: 86611.3047
Epoch [15/20], Loss: 105449.5562
Validation Loss: 87146.5781
Epoch [16/20], Loss: 105527.8594
Validation Loss: 86198.4766
Epoch [17/20], Loss: 104264.2594


2024/11/10 20:10:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Training complete!


test predicion

In [112]:
test_df = test_df.drop(['x', 'y'], axis=1)
test_df = test_df.drop(['severity', 'condition', 'level', 'series_id', 'missing_image'], axis=1)


In [113]:
print(test_df.to_string(index=False, header=True)) 

  study_id                                            image_path
2775207739  data/train_images_origin/2775207739/3249541180/8.dcm
 664153360  data/train_images_origin/664153360/1076245514/13.dcm
2273432465   data/train_images_origin/2273432465/114306451/9.dcm
3777149998  data/train_images_origin/3777149998/3550597941/8.dcm
3967802493 data/train_images_origin/3967802493/2054070341/11.dcm
2780118855 data/train_images_origin/2780118855/2064060968/14.dcm
 247968996   data/train_images_origin/247968996/256361821/14.dcm
 957176622   data/train_images_origin/957176622/1889022706/1.dcm
4075603869  data/train_images_origin/4075603869/2921580289/3.dcm
3605654232    data/train_images_origin/3605654232/26446529/3.dcm
1839242409 data/train_images_origin/1839242409/3211434109/13.dcm
 779162887  data/train_images_origin/779162887/2596438821/18.dcm
 886995462  data/train_images_origin/886995462/3051597267/16.dcm
2348702073   data/train_images_origin/2348702073/323466396/5.dcm
3550756125  data/train_im

predicting

In [ ]:
test_data = test_df

In [ ]:
# calculate prbabilities and probability list
test_dataset = MRIDataset(data=test_data, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Inference loop with probability extraction
results = []
probabilities_list = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)  # Ensure labels are also on the correct device
        outputs = model(images)
        probabilities = torch.softmax(outputs, dim=1)  # Calculate class probabilities
        _, predicted_classes = torch.max(outputs, 1)
        
        # Append predictions and probabilities
        results.append(predicted_classes.item())
        probabilities_list.append(probabilities.cpu().numpy())

